In [66]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score
import numpy as np
import time

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords_pt = stopwords.words('portuguese')

import pandas as pd
import json


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mathe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
add = ['<não há comentários do consumidor>', 'consumidor', 'comentários']
for i in add:
    stopwords_pt.append(i)
print(stopwords_pt)

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [9]:
with open('dados2025.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
df = pd.json_normalize(data)
df.head()

,id,empresa,data,local,status,relato,resposta,nota,comentario
0,1,LG Electronics,2024-01-07,Jacuí - MG,Não Resolvido,Com aproximadamente 3 anos de uso deu um probl...,"Bom dia, Gabriel. \n\nCompreendemos o seu rela...",1,"Como relatei, tenho certeza de que se trata de..."
1,2,Hurb - Hotel Urbano,2024-01-07,Imperatriz - MA,Não Resolvido,Solitei desde 2022 o reembolso o qual não obti...,Finalizamos a sua reclamação e você pode encon...,1,Empresa usa apenas mensagens automáticas e não...
2,3,Hurb - Hotel Urbano,2024-01-07,Três Corações - MG,Não Resolvido,No ano de 2021 comprei um pacote de viagem com...,"Olá, viajante! Tudo bem com você?\n\nFinalizam...",1,Mais uma vez não conseguiram resolver o meu pr...
3,4,Latam Airlines (Tam),2024-01-07,Maringá - PR,Resolvido,Estava programado para eu ir em um voo GRU-MOC...,"Olá ,\n\nInformamos que providenciamos os devi...",5,Finalizaram depositand o valor na Latam Wallet.
4,5,Hurb - Hotel Urbano,2024-01-07,Maringá - PR,Não Resolvido,Viagem comprada para 2023. Pedi reembolso. Dis...,"Olá, viajante! Tudo bem com você?\n\nFinalizam...",1,Respostas protocolares como sempre fizeram sem...


In [10]:
#Conta quantos dos dados sem comentario existem.
num_n_comentarios = df['comentario'].str.count("<não há comentários do consumidor>").sum()
print(num_n_comentarios)

58729


In [11]:
# Cria um filtro para somente os comentarios validos, aqueles que existem, e transoforma a nota para um numero inteiro.
df = df[(df['comentario'] != '<não há comentários do consumidor>') & (df['nota'].isin(['1', '2', '3', '4', '5']))]
df['nota'] = df['nota'].astype(int)

# Transforma os status em numeros, para melhor leitura computacional no treino.
df['status'] = df['status'].map({'Resolvido': 1, 'Não Resolvido': 0})

In [16]:
# A variavel vetorizada vai transformar a seçao de comentario em vetores numericos com base em TF-IDF (term frequency–inverse document frequency),
# Ela vai remover as stopwords e selecionar no maximo 500 palavras mais importantes com base no TF-IDF
vetor = TfidfVectorizer(stop_words=stopwords_pt, max_features=500)
X_text = vetor.fit_transform(df['comentario'])

In [27]:
# Transforma a matriz para um Df, e transforma tudo na coluna para um Str
X_text_df = pd.DataFrame(X_text.toarray())
X_text_df.columns = X_text_df.columns.astype(str)
# Concatena o vetor e a coluna status
X = pd.concat([X_text_df, df['status'].reset_index(drop=True)], axis=1)
y = df['nota']


In [30]:
# Proporçao de classes
proporcao_classe = y.value_counts(normalize = True)
print(f'Proporçao de classes \n{proporcao_classe}')

Proporçao de classes 
nota
1    0.443888
5    0.403791
4    0.062592
3    0.055633
2    0.034096
Name: proportion, dtype: float64


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42)

In [32]:
print(f'Dados de treino e teste:\n{X_train.shape}{X_test.shape}')

Dados de treino e teste:
(116117, 501)(29030, 501)


In [34]:
# Como as notas vao de 1 a 5 esperasse que y_train.nunique() tenha 5 classes.
print(f'Numero de classes:\n{y_train.nunique()}')

Numero de classes:
5


In [35]:
# Criar hiperparametros para nossa randomforest
parametros = {
    'n_estimators' : [25, 50, 100, 200],
    'max_depth': [5,10,20,40, None]
}


In [ ]:
# Implementar Randomforest
model = RandomForestClassifier(random_state= 42, n_jobs= -1)
grid_search = GridSearchCV(model, parametros, cv = 5, scoring='accuracy')
# Marcar o tempo percorrido para o Gridsearch
start = time.time()
print('Iniciando busca')
grid_search.fit(X_train,y_train)

In [41]:
# Pegar o melhor estimator.
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)

In [42]:
print(f'Modelo com maior acurácia: {accuracy*100 :.2f}%')
print(f'Acurácia balanceado do melhor modelo{balanced_accuracy*100:.2f}%')
print(f'Melhor parametros:', grid_search.best_params_)

Modelo com maior acurácia: 80.76%
Acurácia balanceado do melhor modelo38.25%
Melhor parametros: {'max_depth': 40, 'n_estimators': 200}


In [46]:
# Features mais importantes
importantes = best_model.feature_importances_
indices = importantes.argsort()[::-1]
print('Features importantes:')
for i in range(10):
    print(f'{X.columns[indices[i]]}: {importantes[indices[i]]}')
end = time.time()
print(f'Tempo percorrido: {(end - start)/60:.2f} minutos')

Features importantes:
status: 0.43260579805555344
294: 0.042985419401126844
25: 0.025691736157439443
59: 0.021404169426208108
293: 0.021184394232334553
275: 0.015854225061799912
400: 0.01233751875577134
411: 0.012008561607972045
190: 0.011879447508426365
412: 0.0115546896927917
Tempo percorrido: 27.22 minutos


In [55]:
# Palavras com maior peso para avaliaçao:
palavras = vetor.get_feature_names_out()
print(palavras[294])
print(palavras[25])
print(palavras[59])
print(palavras[293])
print(palavras[275])

obrigado
agradeço
atendimento
obrigada
nada


In [70]:
# Salva os resultados
results = pd.DataFrame({
    'Valor real': y_test,
    'Previsão': y_pred,
})

results['Diferença'] = results['Valor real'] - results['Previsão']
results['Correto'] = results['Diferença'].apply(lambda x: 'Sim' if x == 0 else 'Não')
results.to_excel('True_values_X_Predictions.xlsx', index= True)